In [1]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(category=FutureWarning, action="ignore")

In [2]:
price_df = pd.read_csv("data/price.csv")
price_df = price_df.rename(columns={"TIME": "ds", "PRICE": "y"})
price_df['ds'] = pd.to_datetime(price_df['ds'])

print(price_df.head())
print(price_df.shape)

                   ds      y
0 2024-01-01 00:00:00  63.33
1 2024-01-01 00:15:00  60.02
2 2024-01-01 00:30:00  56.71
3 2024-01-01 00:45:00  53.40
4 2024-01-01 01:00:00  50.09
(35133, 2)


In [3]:
METRICS = ["MAE", "RMSE"]
METRICS_VAL = ["MAE_val", "RMSE_val"]

def cv_train(df, lags, forecast_horizon, epochs=100):
    folds = NeuralProphet(n_lags=lags, n_forecasts=forecast_horizon)\
        .crossvalidation_split_df(df, freq='15min', k=10)

    metrics_train = pd.DataFrame(columns=METRICS)
    metrics_test = pd.DataFrame(columns=METRICS_VAL)

    n_fold = 1
    for df_train, df_test in folds:
        print(f"Fold {n_fold}/{len(folds)}")
        n_fold += 1
        predictor = NeuralProphet(n_lags=lags, n_forecasts=forecast_horizon, batch_size=32, epochs=epochs)
        train = predictor.fit(df=df_train, freq="15min", validation_df=df_test)
        test = predictor.test(df=df_test)
        metrics_train = metrics_train.append(train[METRICS].iloc[-1])
        metrics_test = metrics_test.append(test[METRICS_VAL].iloc[-1])

    print(metrics_test.describe().loc[["mean", "std", "min", "max"]])

    return metrics_train, metrics_test

def train(df, lags, forecast_horizon, epochs=100):
    predictor = NeuralProphet(n_lags=lags, n_forecasts=forecast_horizon, batch_size=32, epochs=epochs)
    train_res = predictor.fit(df=df, freq="15T")
    return predictor, train_res


In [4]:
forecast_steps = [1, 8, 12]
n_lags = 24
forecast_horizon = 12

predictor, train_res = train(price_df, n_lags, forecast_horizon, epochs=100)

train_res

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency 15T corresponds to 99.997% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - 15T
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.


Finding best initial lr:   0%|          | 0/263 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

,MAE,RMSE,Loss,RegLoss,epoch
0,274.561737,346.260925,1.210146,0.0,0
1,122.078537,158.823105,0.391553,0.0,1
2,62.707218,79.654106,0.124194,0.0,2
3,32.698212,42.033096,0.036935,0.0,3
4,17.000448,22.675827,0.010769,0.0,4
...,...,...,...,...,...
95,7.078722,10.974842,0.002532,0.0,95
96,7.076402,10.964330,0.002531,0.0,96
97,7.078223,10.974001,0.002531,0.0,97
98,7.076868,10.973886,0.002530,0.0,98


In [5]:
from neuralprophet.utils import save
save(predictor, "price_predictor.np")

In [ ]:
forecasts = predictor.predict(price_df)
predictor.plot(forecasts)

forecasts[[f"yhat{step}" for step in forecast_steps]]